# Classifying Survivors: An Exercise in Machine Learning with Python
**Michael Sieviec**

**August 8, 2019**

# Overview
This point of this notebook is to explore data cleaning, analysis, visualization, engineering, and classification algorithms with the Titanic dataset on Kaggle. I found [Titanic Top 4% with ensemble modeling](https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling) by **Yassine Ghouzam, PhD** to be a helpful resource.

# A look at the data

In [ ]:
import numpy as np
import pandas as pd
from statistics import mode

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

full_data = pd.concat([train.drop(columns = 'Survived'), test], 
                       axis = 0, 
                       sort = False)

full_data.head()

In [ ]:
full_data.describe()

In [ ]:
full_data.shape, train.shape

We see the data is fairly slim: there are only 11 variables and just over 1300 data points, nearly 900 of which come from the training set. Having a smaller test set is fortunate as it gives us more data to build our models on.

In [ ]:
len(full_data['PassengerId'].unique())

Additionally, the `PassengerId` column is fully unique and thus useless to us. `Name` and `Ticket` also appear troublesome, but they could be of use yet.

# Missing Data
Before we continue, we want to eliminate whatever missing values we can.

In [ ]:
## missing
full_data.isnull().sum()

That is quite a few missing in `Cabin`. `Age`, `Fare`, and `Embarked` will be easy: we will fill the first two by the means as grouped by `Sex` and `Pclass`.

In [ ]:
# fill age by means of sex, class
full_data[['Age','Fare']] = (full_data
                    .groupby(['Sex', 'Pclass'])[['Age','Fare']]
                    .transform(lambda x: 
                        x.fillna(round(np.mean(x)))))

full_data['Embarked'].unique()

`Embarked` we will simply fill with the most common value as there are only 3 choices.

In [ ]:
# impute most common port
full_data['Embarked'] = (full_data.Embarked
                         .fillna(mode(full_data.Embarked)))

This leaves `Cabin`, but we will be doing something a little different as seen in the next section.

# New Variables
We have some problem variables, specifically `Cabin`, `Name`, and `Ticket`.

### `Deck`

In [ ]:
full_data['Cabin'].unique()

There are a lot of variables in `Cabin`, but they're not entirely unique and there is a pattern. Nearly every entry is one letter from A-G (and one T) followed by 2 numbers. After referencing some images ([1](https://ssmaritime.com/Titanic-3.htm), [2](https://commons.wikimedia.org/wiki/File:Titanic_cutaway_diagram.png#/media/File:Titanic_cutaway_diagram.png)) of the layout of the Titanic, I assume that the letters represent a deck and the numbers represent a cabin. It is concerning that there are more than 1000 missing values, but we will simply mark these entries with an X and hope the information gained is valuable. A new variable, `Deck`, will consist of the last letter in each string.

In [ ]:
# deck (if any)
full_data['Deck'] = (full_data.Cabin
                     .str.extract('([A-Z])(?=\d*$)', 
                                  expand = True)
                     .fillna('X'))

### `Cabin`
We're also going to extract the last series of numbers in each string, just in case they are somehow useful, and assign them back to `Cabin`. 0s will fill the empty values.

In [ ]:
# cabin letter (if any)
full_data['Cabin'] = (full_data.Cabin
                      .str.extract('(\d+)$', expand = True)
                      .fillna(0))
full_data['Cabin'] = full_data['Cabin'].apply(int) # str to int

### `Title`
In the similarly intimidating `Name` variable, there are discernible reoccurances. Particularly, there appear to be titles, e.g. Mr., Mrs, and so forth. After digging through the names, I found a few that appeared to occur frequently enough to warrant their own designations, as well as some that seemed to warrant aggregation into either an 'Officer' or 'Other' category.

In [ ]:
# title
title_extract = ('(Mr\.|Master|Mrs\.|Miss\.|Rev\.|Capt\.|Col\.|Major\.)')
title_dict = {'Capt\.|Major\.|Col\.' : 'Officer'}
full_data['Title'] = (full_data.Name
                      .str.extract(title_extract)
                      .replace(title_dict, 
                               regex = True)
                      .fillna('Other'))

In [ ]:
full_data.groupby('Title').size()

That looks a lot more manageable than `Name`.

### `TicketNumber`
Less frequently explored seems to be the `Ticket` variable. It is understandable, as it's arguably the messiest of all the variables. However, I did catch on to one possible pattern.

In [ ]:
full_data['Ticket'].unique()

There appears to be a number in every entry. I don't know that this will be useful yet, but we have precious little information so I'm going to extract these anyway and hope they come in handy. There also do appear to be some patterns in the lettering, but they aren't uniform enough for me to want to explore them.

In [ ]:
# ticketnumber
full_data['TicketNumber'] = (full_data.Ticket
                             .str.extract('(\d+)$', expand = True))
full_data.loc[full_data.TicketNumber.notnull(), 'TicketNumber'] = (full_data
                                                                   .loc[full_data.TicketNumber.notnull(),
                                                                        'TicketNumber']
                                                                   .astype(int)) # str to int

# 4 missing tickets
(full_data.Ticket
 .str.extract('(\d+)$', expand = True)
 .isnull()
 .sum())

And we finally impute these missing ticket numbers from the data as grouped by `Pclass` and `Embarked`.

In [ ]:
full_data['TicketNumber'] = (full_data
                             .groupby(['Pclass', 'Embarked'])['TicketNumber']
                             .transform(lambda x: x.fillna(np.mean(x))))

# Visualizations
The data is now clean enough to have a look at graphically. We're looking for insight into what makes someone more likely to survive.

In [ ]:
# new training set for probabilities and visualizations
train_for_vis = pd.concat([train.Survived, full_data[:train.shape[0]]], axis = 1)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette('cool')

# without family more likely to die
sns.catplot(x = 'Parch', 
            y = 'Survived', 
            data = train_for_vis,
            height = 5,
            aspect = 1.5,
            kind = 'bar')
plt.title('Figure 1: Survival by Number of Parents/Children')

It appears that people travelling in small families faired better than either those who were alone or with large families.

In [ ]:
sns.catplot(x = 'SibSp', 
            y = 'Survived', 
            data = train_for_vis,
            height = 5,
            aspect = 1.5,
            kind = 'bar')
plt.title('Figure 2: Survival by Number of Siblings/Spouses')

The pattern holds for those with siblings and spouses.

In [ ]:
# survival by deck
sns.catplot(x = 'Deck', 
            y = 'Survived', 
            data = train_for_vis, 
            kind = 'bar',
            height = 5,
            aspect = 1.85,
            order = sorted(list(train_for_vis['Deck'].unique())))
plt.title('Figure 3: Survival by Deck')

`Deck` turned out to be insightful: those on decks marked A, G, or T, or those without a marked deck (X) all fared significantly worse than those on others, with passengers on deck C falling somewhere in between.

In [ ]:
# survival by sex
sns.catplot(x = 'Sex', 
            y = 'Survived', 
            data = train_for_vis,
            kind = 'bar')
plt.title('Figure 4: Survival by Sex')

It looks like the men were really out of luck on this trip.

In [ ]:
# survival by age
g = sns.FacetGrid(train_for_vis, 
                  hue = 'Survived',
                  height = 5,
                  aspect = 1.5,
                  palette = 'cool')
g.map(sns.kdeplot, 'Age', shade = True).add_legend()
plt.title('Figure 5: Survival by Age')

Here's a compelling figure: it shows that there are pretty clear discrepancies in survival rates based on age groups. We will make use of them.

In [ ]:
g = sns.FacetGrid(train_for_vis, 
                  hue = 'Survived',
                  height = 5,
                  aspect = 1.5,
                  palette = 'cool')
g.map(sns.kdeplot, 'TicketNumber', shade = True).add_legend()
plt.title('Figure 6: Survival by TicketNumber')

And another: there appear to be 3 distinct classes of survival rates in the ticket numbers. We'll make use of this, as well.

In [ ]:
g = sns.FacetGrid(train_for_vis, 
                  hue = 'Survived',
                  height = 5,
                  aspect = 1.5,
                  palette = 'cool')
g.map(sns.kdeplot, 'Cabin', shade = True).add_legend()
plt.title('Figure 7: Survival by Cabin Number')

I don't know how useful this one is going to be, but let's keep exploring.

### Correlation
Correlation is one method of discerning if a variable has a relationship with another or not. For our purposes, we're let's look at our variables' correlation with survival.

In [ ]:
corr_map = train_for_vis.corr(method = 'pearson')
corr_map = (corr_map
            .transform(lambda x: np.flip(x, 0), 
                       axis = 0))
mask = np.tri(corr_map.shape[0], k = -1).T

plt.figure(figsize = (12, 9))
sns.heatmap(corr_map, 
            annot = True, 
            cmap = 'GnBu_r', 
            mask = np.flip(mask, axis = 1),
            vmin = 0,
            vmax = 1)
plt.title("Figure 8: Pearson Correlation Coefficient")

As expected, `PassengerId` is not useful. Interestingly, `TicketNumber` is slightly more correlated than `Age` is. And actually, so is `Cabin`, so I guess we're going to hang on to that one.

However, the Pearson correlation coefficient only detects linear relationships between the variables. These variables may not be linearly related. Let's see if Spearman's rho is any more enlightening.

In [ ]:
corr_map = train_for_vis.corr(method = 'spearman')
corr_map = (corr_map
            .transform(lambda x: np.flip(x, 0), 
                       axis = 0))
mask = np.tri(corr_map.shape[0], k = -1).T

plt.figure(figsize = (12, 9))
sns.heatmap(corr_map, 
            annot = True, 
            cmap = 'GnBu_r', 
            mask = np.flip(mask, axis = 1),
            vmin = 0,
            vmax = 1)
plt.title("Figure 9: Spearman's Rho")

The correlations for the relavant variables are all a little stronger when we look at things non-linearly. Not astoundingly so, but better at least.

# Variable Engineering
There are still some things we can do to improve the predicting power of our variables. For my own curiosity, I'm going to explore ordinal categorization of our variables as much as is reasonable (and maybe a little that isn't, but hey, messing around is fun). The idea behind this is that besides things like your title or accomodations just being a factor in your survival, we can discern mathematically the chances of your survival when each of these is taken individually. If you sort these chances by order&mdash;voilà!&mdash;you have ordinal or qualitative data. This may not be the optimal approach but it sounds fun and makes enough sense to me, so I'm doing it.

### `Sex`
First, we make sex a binary variable ordered based on the fact that women clearly had better survival outcomes than men.

In [ ]:
full_data['Sex'] = (full_data['Sex']
                    .map({'male' : 0, 'female' : 1}))

### `TicketCat`
Next, we categorize `TicketNumber` based on the survival discrepancies from the earlier graph.

In [ ]:
def ticket_transform(data):
    if data < 200000:
        data = 0
    elif 200000 <= data < 1000000:
        data = 1
    else:
        data = 2
    return data

full_data['TicketCat'] = (full_data['TicketNumber']
                          .transform(ticket_transform))

Now we actually order them by survival probabilities.

In [ ]:
train_for_vis = pd.concat([train.Survived, full_data[:train.shape[0]]], axis = 1)

# make ordinal
ticket_by_prob = (train_for_vis
                 .groupby(['TicketCat'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
ticket_map = dict(zip(ticket_by_prob,
                    [x for x in range(len(ticket_by_prob))]))
full_data['TicketCat'] = full_data['TicketCat'].map(ticket_map)

In [ ]:
(train_for_vis
    .groupby(['TicketCat'])['Survived']
    .agg('mean')
    .sort_values(ascending = True))

We see that the category of highest ticket numbers had the worst survival outcomes by a small margin.

We proceed in a similar manner for other variables.

###  `AgeCat`
`Age` as categorized by survival.

In [ ]:
def age_transform(data):
    if data < 17:
        data = 0
    elif 17 <= data < 32:
        data = 1
    elif 32 <= data < 42:
        data = 2
    elif 42 <= data < 60:
        data = 3
    else:
        data = 4
    return data

full_data['AgeCat'] = (full_data['Age']
                       .transform(age_transform))
# make ordinal
train_for_vis = pd.concat([train.Survived, full_data[:train.shape[0]]], axis = 1)
age_by_prob = (train_for_vis
                 .groupby(['AgeCat'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
age_map = dict(zip(age_by_prob,
                    [x for x in range(len(age_by_prob))]))
full_data['AgeCat'] = full_data['AgeCat'].map(age_map)

### `FamilyCat`
And family size category.

In [ ]:
full_data['FamilySize'] = full_data['SibSp'] + full_data['Parch']

full_data['FamilyCat'] = (full_data['FamilySize']
                          .map({0 : 'Alone',
                                **dict.fromkeys([1, 2, 3], 'Small'),
                                **dict.fromkeys([4, 5, 6, 7, 8, 9, 10], 'Large')}))

train_for_vis = pd.concat([train.Survived, full_data[:train.shape[0]]], axis = 1)

# ordinal
fam_by_prob = (train_for_vis
                 .groupby(['FamilyCat'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
fam_map = dict(zip(fam_by_prob,
                    [x for x in range(len(fam_by_prob))]))
full_data['FamilyCat'] = full_data['FamilyCat'].map(fam_map)

### `Title`

In [ ]:
# ordinal title
title_by_prob = (train_for_vis
                 .groupby(['Title'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)

title_map = dict(zip(title_by_prob,
                     [x for x in range(len(title_by_prob))]))
full_data['Title'] = full_data['Title'].map(title_map)

### `Pclass`

In [ ]:
# ordinal class
class_by_prob = (train_for_vis
                 .groupby(['Pclass'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
class_map = dict(zip(class_by_prob,
                     [x for x in range(len(class_by_prob))]))
full_data['Pclass'] = full_data['Pclass'].map(class_map)

### `Embarked`

In [ ]:
# ordinal port
port_by_prob = (train_for_vis
                 .groupby(['Embarked'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
port_map = dict(zip(port_by_prob,
                    [x for x in range(len(port_by_prob))]))
full_data['Embarked'] = full_data['Embarked'].map(port_map)

### `Deck`

In [ ]:
# ordinal deck
deck_by_prob = (train_for_vis
                 .groupby(['Deck'])['Survived']
                 .agg('mean')
                 .sort_values(ascending = True)
                 .index)
deck_map = dict(zip(deck_by_prob,
                    [x for x in range(len(deck_by_prob))]))
full_data['Deck'] = full_data['Deck'].map(deck_map)


And we dispose of the old variables in favor of the re-engineered ones.

In [ ]:
to_drop_cols = ['PassengerId', 'Name', 'Age',
                'SibSp', 'Parch', 'Ticket', 
                'TicketNumber', 'FamilySize']

full_data = full_data.drop(columns = to_drop_cols)
full_data.head()

That's looking much easier to deal with, but we've got a little further to go.

# Outliers and Insight
Outliers can mess up any model, so let's see if we can't find any in a pair plot. We're also going to take this time to inspect each variable combination to discern if there is some kind of clear classification boundary between those who survived and those who didn't in order to guide our model selection.

In [ ]:
train_clean = pd.concat([train['Survived'], 
                         full_data[:train.shape[0]]], axis = 1)
sns.pairplot(train_clean, 
             hue = 'Survived', 
             vars = list(train_clean.drop(columns = 'Survived').columns))
plt.suptitle('Figure 10: Pairplot of All Variables', y = 1.02)

Though the scale is difficult to work with, there doesn't appear to be a clear boundary between those who survived and those who didn't in any of the plots, so I won't employ discriminant analysis. We do, however, see some outliers in the `Fare`/`Cabin` graph: the highest fares are way above all of the others. Those should be easy to find and be rid of.

In [ ]:
to_drop_rows = [343, 679, 258, 737] # high fares

# Normalization
`Fare` and `Cabin` are *heavily* skewed, we can tell just by the exploratory graphs above, so we're going to log(1+x) transform them so that they're a little more normal because we want to use at least one generalized linear model.

In [ ]:
full_data[['Cabin', 'Fare']] = (full_data[['Cabin', 'Fare']]
                                .transform(lambda x: np.log(1+x)))

In [ ]:
full_data.head()

Ok, now we reassemble our training and testing sets.

In [ ]:
train_clean = pd.concat([train['Survived'], 
                         full_data[:train.shape[0]]], axis = 1)
train_clean = train_clean.drop(index = to_drop_rows)
test_clean = full_data[(train_clean.shape[0] + len(to_drop_rows)):]
train_clean.shape, test_clean.shape  # minus 4 outliers

In [ ]:
train.shape, test.shape

Let's have a look at a correlation matrix with our new variables.

In [ ]:
corr_map = train_clean.corr(method = 'spearman')
corr_map = (corr_map
            .transform(lambda x: np.flip(x, 0), 
                       axis = 0))
mask = np.tri(corr_map.shape[0], k = -1).T

plt.figure(figsize = (12, 9))
sns.heatmap(corr_map, 
            annot = True, 
            cmap = 'GnBu_r', 
            mask = np.flip(mask, axis = 1),
            vmin = 0,
            vmax = 1)
plt.title("Figure 11: Spearman's Rho")

Our new variables seem to be better predictors of `Survived` than the old. Notice how highly correlated `Title`/`Sex` and `Cabin`/`Deck` are. The first relationship is no surprise&mdash;in those days, your title would be in direct relation to your sex. The second is a little more compelling, as `Cabin` became the cabin numbers, and `Deck` the cabin letters by increasing order of survival. This seems to mean that as your chances of your living on a better deck in terms of survival increased, the chances of you having a higher cabin number did, also, and significantly so. There *were* a lot of zeros imputed into `Cabin`. I would not be surprised to find that the absense of a cabin number in the first place were not a mistake, and that many simply didn't have accomodations considered as a cabin.

# Modeling
Now we explore a few models. I won't be doing ensembling (at least, not meta-ensembling), but simply sticking to some methods I wanted to get more comfortable with as well as explore one or two that I'm unfamiliar with.

Let's set up the shop:

In [ ]:
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score
from collections import namedtuple
from sklearn.utils import shuffle

shuffled = shuffle(train_clean, random_state = 42)
predictors = shuffled.drop(columns = 'Survived')
response = shuffled['Survived']

# accuracy assessment
def get_accuracy(model, resp):
    out = namedtuple('Output', 'Accuracy Predictions')
    pred = cross_val_predict(model, predictors, resp, cv = 10)
    acc = accuracy_score(resp, pred)
    return out(acc, pred)

A function for assessing the model out-of-sample performance was defined based on a cross-validation loop, and the training data shuffled because some of the classifiers we will use take subsets of the training data at each step, so we wanted to break up any patterns in the data that may have been present in any given chunk(s).

# Logistic Regression
This one is a (modern) classic. It is a generalized linear model that maximizes a [log-likelihood function](https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc) to assign probabilities to each observation. While comparatively simple to some other methods, I once tried to program it from scratch for a school project and found it fantastically difficult. Still, the nuts and bolts are easy to understand: if the probability for an observation is < 0.5, a 0 is predicted&mdash;else, a 1.

Note: somewhat extensive parameter tuning was done with most of the models which I will not reiterate here.

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(C = 1.75,
                              solver = 'liblinear',
                              fit_intercept = True, 
                              max_iter = 10000)
acc_lr = get_accuracy(model_lr, response)
print(f'Logistic Regression classification accuracy was {acc_lr.Accuracy}')

For a relatively traditional model, that's not bad at all. It's not great, but it's not bad.

In [ ]:
model_lr.fit(predictors, response)
print('Logistic Regression Coefficients')
features = pd.DataFrame(dict(zip(list(train_clean.drop(columns = 'Survived')), 
                      list(model_lr.coef_[0]))), index = range(1)).sort_values(by = 0, axis = 1, ascending = False)
sns.catplot(data = features,
            kind = 'bar',
            height = 5,
            aspect = 1.5,
            palette = 'magma_r')
plt.title('Figure 12: Logistic Regression Coefficients')

We see logistic regression more positively associated `Class`, `Title`, and `FamilyCat` with survival.

# Gaussian Process Classification
This one caught my eye in the sklearn catalog. After reading some of [this book excerpt](http://www.gaussianprocess.org/gpml/chapters/RW3.pdf) (Rasmussen and Williams) and learning it is a more generalized version of logistic regression, I decided I'd try it.

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
model_gpc = GaussianProcessClassifier(n_restarts_optimizer = 10,
                                      max_iter_predict = 100000,
                                      random_state = 5)
acc_gpc = get_accuracy(model_gpc, response)
print(f'Gaussian Process classification accuracy was {acc_gpc.Accuracy}')

Unsurprisingly, it does a little better than traditional logistic regression given it's more general nature.

# K-Nearest Neighbors
This one is another classic of the classifiers. It relies on Euclidean distance by assigning the mode class of the k-nearest neighbors to an unclassified data point. Sklearn also incorporates some more complex methods to help cut down on computation time (as this process can be quite expensive with larger datasets).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_kn = KNeighborsClassifier(n_neighbors = 9,
                                n_jobs = 2)
acc_kn = get_accuracy(model_kn, response)
print(f'K-Nearest Neighbors classification accuracy was {acc_kn.Accuracy}')

Getting a little better still.

# Support Vector Classification
[A Practical Guide to Support Vector Classiﬁcation](https://www.researchgate.net/profile/Chenghai_Yang/publication/272039161_Evaluating_unsupervised_and_supervised_image_classification_methods_for_mapping_cotton_root_rot/links/55f2c57408ae0960a3897985/Evaluating-unsupervised-and-supervised-image-classification-methods-for-mapping-cotton-root-rot.pdf) (Hsu et. al.) does a better job of explaining how these work than I probably ever will.

In [ ]:
from sklearn.svm import SVC
model_svc = SVC(max_iter = 1000000, 
                C = 0.5,
                kernel = 'rbf',
                gamma = 'scale',
                probability = True,
                random_state = 5)
acc_svc = get_accuracy(model_svc, response)
print(f'Support Vector classification accuracy was {acc_svc.Accuracy}')

# XGBoost
XGBoost is a boosted trees algorithm. They do a great job of explaining the inner workings of them [here](https://xgboost.readthedocs.io/en/latest/tutorials/model.html).

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(reg_alpha = .35,
                              reg_lambda = .6,
                              nthread = 2,
                              seed = 1)
acc_xgb = get_accuracy(model_xgb, response)
print(f'XGBoost classification accuracy was {acc_xgb.Accuracy}')

In [ ]:
model_xgb.fit(predictors, response)
model_xgb.feature_importances_
features = pd.DataFrame(dict(zip(list(train_clean.drop(columns = 'Survived')), 
                      list(model_xgb.feature_importances_))), index = range(1)).sort_values(by = 0, axis = 1, ascending = False)
sns.catplot(data = features,
            kind = 'bar',
            height = 5,
            aspect = 1.5,
            palette = 'magma_r')
plt.title('Figure 13: XGBoost Feature Importances')

The three most important features are the same for XGBoost as the logit model.

# Random Forest
Random forest is a very popular machine learning technique based on the aggregation of decision trees in a manner similar but not identical to XGBoost.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state = 5,
                                  max_depth = 6,
                                  max_features = 8,
                                  n_estimators = 500,
                                  n_jobs = 2)
acc_rf = get_accuracy(model_rf, response)
print(f'Random forest classification accuracy was {acc_rf.Accuracy}')

In [ ]:
model_rf.fit(predictors, response)
model_rf.feature_importances_
features = pd.DataFrame(dict(zip(list(train_clean.drop(columns = 'Survived')), 
                      list(model_rf.feature_importances_))), index = range(1)).sort_values(by = 0, axis = 1, ascending = False)
sns.catplot(data = features,
            kind = 'bar',
            height = 5,
            aspect = 1.5,
            palette = 'magma_r')
plt.title('Figure 14: Random Forest Feature Importances')

As with XGBoost, random forest found `Title` to be the most important feature. Interestingly, it rounded out the top three most important features with `Fare` and `Sex`, unlike the others we were able to visualize, and it performed the best by a small margin.

# Summary
The data required a fair bit of cleaning and engineering, but it was ultimately very usable, which is a testament to being creative in your approach. For prediction, random forest (unsurprisingly) did the best, clocking in at greater than 84% accuracy during cross-validation. Interestingly, feature coefficients and importance fell not very far from our Spearman's rho correlation, so it may be a good starting point for feature selection and engineering after all. Seeing what others have done, this seems to be on par with the better results for individual algorithms. As others have done, one might employ meta modeling to improve accuracy.